# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi
!pip install tqdm

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/sherlin01/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [4]:
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [7]:
#login.items()
#login['API Key']

In [8]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['API Key'], timeout_s = 5.0)

### Define Search Terms and File Paths

In [9]:
# set our API call parameters and filename before the first call
location = 'Fresno, TX 77545'
term = 'wings'

In [11]:
location.split(',')[0]

'Fresno'

In [14]:
## Specify folder for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok = True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [15]:
JSON_FILE

'Data/Fresno-wings.json'

### Check if Json File exists and Create it if it doesn't

In [19]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON folder name is not empty:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
          json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Fresno-wings.json already exists.


In [18]:
os.path.isfile(JSON_FILE)

True

### Load JSON FIle and account for previous results

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [20]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)

In [21]:
type(results)

dict

In [22]:
len(results)

3

In [23]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [24]:
results['total']

202

In [25]:
results['region']

{'center': {'longitude': -95.46981811523438, 'latitude': 29.537609279094596}}

In [26]:
results['businesses']

[{'id': '8f3FMvchTpzoUYUfpPXjtQ',
  'alias': 'flava-wings-houston',
  'name': 'Flava Wings',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Tpt9-iFbO7oI51XBcUOgiw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/flava-wings-houston?adjust_creative=rZiFgfUX1eajVOVygdFykg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=rZiFgfUX1eajVOVygdFykg',
  'review_count': 136,
  'categories': [{'alias': 'chicken_wings', 'title': 'Chicken Wings'}],
  'rating': 4.0,
  'coordinates': {'latitude': 29.581544612414802, 'longitude': -95.4547456},
  'transactions': ['delivery', 'pickup'],
  'price': '$$',
  'location': {'address1': '7395 McHard Rd',
   'address2': 'Unit 102',
   'address3': None,
   'city': 'Houston',
   'zip_code': '77053',
   'country': 'US',
   'state': 'TX',
   'display_address': ['7395 McHard Rd', 'Unit 102', 'Houston, TX 77053']},
  'phone': '+12817212691',
  'display_phone': '(281) 721-2691',
  'distance': 5098.260030095037},
 {'id': 'm

In [27]:
## How many results total?
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8f3FMvchTpzoUYUfpPXjtQ,flava-wings-houston,Flava Wings,https://s3-media2.fl.yelpcdn.com/bphoto/Tpt9-i...,False,https://www.yelp.com/biz/flava-wings-houston?a...,136,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.0,"{'latitude': 29.581544612414802, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '7395 McHard Rd', 'address2': 'Un...",+12817212691,(281) 721-2691,5098.260030
1,mlOelmCRqLOkWlLcAIZ-BA,swamp-chicken-missouri-city,Swamp Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/iV2Jhj...,False,https://www.yelp.com/biz/swamp-chicken-missour...,23,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.0,"{'latitude': 29.545382778092428, 'longitude': ...",[],NaN,"{'address1': '8035 Hwy 6', 'address2': '', 'ad...",+12819698945,(281) 969-8945,7523.026816
2,zcc818cmdAJ6JDiS8ODXAg,big-city-wings-pearland,Big City Wings,https://s3-media2.fl.yelpcdn.com/bphoto/LN4X0h...,False,https://www.yelp.com/biz/big-city-wings-pearla...,286,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"{'latitude': 29.557198, 'longitude': -95.371067}","[delivery, pickup]",$$,"{'address1': '9721 Broadway St', 'address2': N...",+13467545980,(346) 754-5980,9797.744647
3,kBPaHWjREZNgr35J5l3_jg,wingstop-arcola,Wingstop,https://s3-media1.fl.yelpcdn.com/bphoto/2X_qSY...,False,https://www.yelp.com/biz/wingstop-arcola?adjus...,38,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.0,"{'latitude': 29.50753417185821, 'longitude': -...",[],NaN,"{'address1': '5243 Fm 521 Rd', 'address2': 'St...",+12814319464,(281) 431-9464,3479.181431
4,b4a8eGt5rBr8ubKGXO8wYA,take-the-wheel-mobile-fresno,Take The Wheel Mobile,https://s3-media2.fl.yelpcdn.com/bphoto/WWd4yD...,False,https://www.yelp.com/biz/take-the-wheel-mobile...,59,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"{'latitude': 29.523673, 'longitude': -95.45414}","[delivery, pickup]",$,"{'address1': '4233 Fm 521 Rd', 'address2': Non...",+18323342181,(832) 334-2181,2171.094342
5,tsLFO0SuZPjqCtGcDNnWig,bar-kada-missouri-city,Bar Kada,https://s3-media3.fl.yelpcdn.com/bphoto/WEs1ij...,False,https://www.yelp.com/biz/bar-kada-missouri-cit...,49,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",3.5,"{'latitude': 29.537163, 'longitude': -95.533925}",[],NaN,"{'address1': '9009 Sienna Crossing Dr', 'addre...",+17132275232,(713) 227-5232,6197.980513
6,s7urMWCZg99G2AkYbfCZZw,wing-station-missouri-city-3,Wing Station,https://s3-media2.fl.yelpcdn.com/bphoto/sy9AFx...,False,https://www.yelp.com/biz/wing-station-missouri...,28,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.0,"{'latitude': 29.513184, 'longitude': -95.526678}","[delivery, pickup]",NaN,"{'address1': '4340 Sienna Pkwy', 'address2': '...",+13468167213,(346) 816-7213,6135.353721
7,waguhjeCnyyIv_5PkBDcIQ,daves-hot-chicken-missouri-city,Dave's Hot Chicken,https://s3-media1.fl.yelpcdn.com/bphoto/LE-VoO...,False,https://www.yelp.com/biz/daves-hot-chicken-mis...,149,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.0,"{'latitude': 29.56263, 'longitude': -95.562596}","[delivery, pickup]",$$,"{'address1': '6324 Hwy 6 S', 'address2': None,...",+18329874799,(832) 987-4799,9396.704768
8,HjHjZnRqXYE5_jRwV4pPrQ,bonchon-pearland-6,Bonchon,https://s3-media4.fl.yelpcdn.com/bphoto/_JgCWc...,False,https://www.yelp.com/biz/bonchon-pearland-6?ad...,303,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"{'latitude': 29.55288834, 'longitude': -95.394...","[delivery, pickup]",$$,"{'address1': '11200 Broadway St', 'address2': ...",+13464105167,(346) 410-5167,7497.781280
9,xkqqY29BviSEtSZeFzc2cg,big-mammas-home-cooking-missouri-city,Big Mammas Home Cooking,https://s3-media4.fl.yelpcdn.com/bphoto/Cywt_e...,False,https://www.yelp.com/biz/big-mammas-home-cooki...,54,"[{'alias': 'soulfood', 'title': 'Soul Food'}, ...",3.5,"{'latitude': 29.540765589379852, 'longitude': ...","[delivery, pickup]",NaN,"{'address1': '8731 Hwy 6',

- Where is the actual data we want to save?

In [28]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [30]:
(results['total'])/ results_per_page

10.1

In [31]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

11

In [32]:
for i in tqdm_notebook(range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location = location, term = term,
                                   offset = n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print(' [!] ERROR', e)


  0%|          | 0/11 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [33]:
df = pd.read_json(JSON_FILE)

In [34]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,mlOelmCRqLOkWlLcAIZ-BA,swamp-chicken-missouri-city,Swamp Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/iV2Jhj...,False,https://www.yelp.com/biz/swamp-chicken-missour...,23,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.0,"{'latitude': 29.545382778092428, 'longitude': ...",[],"{'address1': '8035 Hwy 6', 'address2': '', 'ad...",+12819698945,(281) 969-8945,7523.026816,NaN
1,zcc818cmdAJ6JDiS8ODXAg,big-city-wings-pearland,Big City Wings,https://s3-media2.fl.yelpcdn.com/bphoto/LN4X0h...,False,https://www.yelp.com/biz/big-city-wings-pearla...,286,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"{'latitude': 29.557198, 'longitude': -95.371067}","[pickup, delivery]","{'address1': '9721 Broadway St', 'address2': N...",+13467545980,(346) 754-5980,9797.744647,$$
2,kBPaHWjREZNgr35J5l3_jg,wingstop-arcola,Wingstop,https://s3-media1.fl.yelpcdn.com/bphoto/2X_qSY...,False,https://www.yelp.com/biz/wingstop-arcola?adjus...,38,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.0,"{'latitude': 29.50753417185821, 'longitude': -...",[],"{'address1': '5243 Fm 521 Rd', 'address2': 'St...",+12814319464,(281) 431-9464,3479.181431,NaN
3,b4a8eGt5rBr8ubKGXO8wYA,take-the-wheel-mobile-fresno,Take The Wheel Mobile,https://s3-media2.fl.yelpcdn.com/bphoto/WWd4yD...,False,https://www.yelp.com/biz/take-the-wheel-mobile...,59,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"{'latitude': 29.523673, 'longitude': -95.45414}","[pickup, delivery]","{'address1': '4233 Fm 521 Rd', 'address2': Non...",+18323342181,(832) 334-2181,2171.094342,$
4,tsLFO0SuZPjqCtGcDNnWig,bar-kada-missouri-city,Bar Kada,https://s3-media3.fl.yelpcdn.com/bphoto/WEs1ij...,False,https://www.yelp.com/biz/bar-kada-missouri-cit...,49,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",3.5,"{'latitude': 29.537163, 'longitude': -95.533925}",[],"{'address1': '9009 Sienna Crossing Dr', 'addre...",+17132275232,(713) 227-5232,6197.980513,NaN


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             201 non-null    object 
 1   alias          201 non-null    object 
 2   name           201 non-null    object 
 3   image_url      201 non-null    object 
 4   is_closed      201 non-null    bool   
 5   url            201 non-null    object 
 6   review_count   201 non-null    int64  
 7   categories     201 non-null    object 
 8   rating         201 non-null    float64
 9   coordinates    201 non-null    object 
 10  transactions   201 non-null    object 
 11  location       201 non-null    object 
 12  phone          201 non-null    object 
 13  display_phone  201 non-null    object 
 14  distance       201 non-null    float64
 15  price          143 non-null    object 
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 23.9+ KB


In [36]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Fresno-wings.csv.gz'

In [37]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [38]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 197,853 Bytes
CSV.GZ FILE: 27,766 Bytes
the csv.gz is 7.125729309227112 times smaller!


## Next Class: Processing the Results and Mapping 